In [1]:
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.io.fits import getdata
from astropy.time import Time
from astropy.io import fits

In [2]:
df = pd.read_csv('csv/full_merged.csv')
df.head()

,SL Target,SEQID,Module,Primary Target,Exposure (s),RA,DEC,PA,Unnamed: 8,In old sheet?,Notes
0,??,10402003002,B,Crab_bkgd_PA330,19232.6,83.1538,21.5296,330.5875,1,N,GR (from Crab?)
1,??,10502003002,B,Crab_bkgd_PA330,28264.7,83.1438,21.5151,329.6554,1,N,SL/GR in both
2,??,10601407002,A,N132D,82687.4,81.1972,-69.6945,336.5505,2,N,"Strong on FPMA only, some LMC source"
3,??,30001012002,B,NuSTARJ163433m473841,16599.9,248.6712,-47.6363,171.9830,1,Y,GR + multiple SL
4,??,30001016002,B,4U1630m47,22071.8,248.5333,-47.3794,164.6452,0,Y,"Corner, prob GX 340+0?"


In [3]:
greenlist = ['SL Target', 'SEQID', 'Module', 'Primary Target', 'Exposure (s)', 'RA',
       'DEC', 'Notes']
for col in df.columns:
    if col not in greenlist:
        df = df.drop(axis=1, labels=col)
        
# Drop everything with NaN in the SL Target column
df = df.dropna(subset=['SL Target'])
df.head()

,SL Target,SEQID,Module,Primary Target,Exposure (s),RA,DEC,Notes
0,??,10402003002,B,Crab_bkgd_PA330,19232.6,83.1538,21.5296,GR (from Crab?)
1,??,10502003002,B,Crab_bkgd_PA330,28264.7,83.1438,21.5151,SL/GR in both
2,??,10601407002,A,N132D,82687.4,81.1972,-69.6945,"Strong on FPMA only, some LMC source"
3,??,30001012002,B,NuSTARJ163433m473841,16599.9,248.6712,-47.6363,GR + multiple SL
4,??,30001016002,B,4U1630m47,22071.8,248.5333,-47.3794,"Corner, prob GX 340+0?"


In [7]:

# Set the "public" cutoff date to be June 1, 2020
cutoff_t = Time('2020-06-01T00:00:00.123456789', format='isot', scale='utc')
cutoff_mjd = cutoff_t.mjd

numaster = Table.read('fits/numaster.fits', format = 'fits')
    
dn = numaster.to_pandas()

greenlist = ['NAME', 'RA', 'DEC', 'TIME', 'END_TIME',
       'OBSID', 'EXPOSURE_A', 'EXPOSURE_B', 'PUBLIC_DATE']
for col in dn.columns:
    if col not in greenlist:
        dn.drop(col, axis=1, inplace=True)

dn['OBSID'] = pd.to_numeric(dn['OBSID'])
dn.columns

dn['PUBLIC'] = np.where((dn['PUBLIC_DATE']<cutoff_mjd)&(dn['PUBLIC_DATE'] > 0), 'yes', 'no')
dn

,NAME,RA,DEC,TIME,END_TIME,OBSID,EXPOSURE_A,EXPOSURE_B,PUBLIC_DATE,PUBLIC
0,b'3C273_pcs11 ...,187.2386,2.0479,56109.875778,56109.941750,15011001,2881.8646,2882.2901,56533,yes
1,b'3C273_pcs12 ...,187.2038,2.0472,56109.941750,56110.011194,15012001,2880.8013,2894.2422,56533,yes
2,b'3C273_pcs13 ...,187.2367,2.0157,56110.011194,56110.080639,15013001,2572.5306,2575.3012,56533,yes
3,b'3C273_pcs14 ...,187.3142,2.0520,56110.080639,56110.146611,15014001,2607.3166,2609.0228,56533,yes
4,b'3C273_pcs15 ...,187.2341,2.1272,56110.146611,56110.212583,15015001,2844.8164,2847.9784,56533,yes
...,...,...,...,...,...,...,...,...,...,...
7357,b'J0612m06 ...,93.0010,-6.3691,-99.000000,-99.000000,60601033,-70000.0000,-70000.0000,-99,no
7358,b'ESO495mG005 ...,124.8114,-25.1879,-99.000000,-99.000000,60668002,-100000.0000,-100000.0000,-99,no
7359,b'ESO420mG013 ...,63.4570,-32.0069,-99.000000,-99.000000,60668003,-100000.0000,-100000.0000,-99,no
7360,b'MRK_1218 ...,129.5456,24.8953,-99.000000,-99.000000,60601034,-75000.0000,-75000.0000,-99,no


In [8]:

df['TIME'] = [-1. for x in range(len(df['SEQID']))]
df['END_TIME'] = [-1. for x in range(len(df['SEQID']))]

for si in np.unique(df['SEQID']):
#    print(dn.loc[dn['OBSID'] == si, 'TIME'].values[0])
    df.loc[df['SEQID'] == si, 'TIME'] = dn.loc[dn['OBSID'] == si, 'TIME'].values[0]
    df.loc[df['SEQID'] == si, 'END_TIME'] = dn.loc[dn['OBSID'] == si, 'END_TIME'].values[0]
df.head()

,SL Target,SEQID,Module,Primary Target,Exposure (s),RA,DEC,Notes,TIME,END_TIME
0,??,10402003002,B,Crab_bkgd_PA330,19232.6,83.1538,21.5296,GR (from Crab?),58156.445245,58156.889690
1,??,10502003002,B,Crab_bkgd_PA330,28264.7,83.1438,21.5151,SL/GR in both,58518.459134,58519.122329
2,??,10601407002,A,N132D,82687.4,81.1972,-69.6945,"Strong on FPMA only, some LMC source",58921.434829,58923.129273
3,??,30001012002,B,NuSTARJ163433m473841,16599.9,248.6712,-47.6363,GR + multiple SL,56374.354944,56374.764666
4,??,30001016002,B,4U1630m47,22071.8,248.5333,-47.3794,"Corner, prob GX 340+0?",56722.955639,56723.486889


In [9]:
obj_type = pd.read_csv('csv/target_info.csv')
green_list = ['Confirmed Sources', 'Type']
for col in obj_type.columns:
    if col not in green_list:
        if col in obj_type.columns:
            obj_type = obj_type.drop(axis=1,labels=col)
obj_type.head()

,Confirmed Sources,Type
0,2MASX J09172716-6456271,AGN
1,3C 273,AGN
2,4U 0614+091,LMXB-NS
3,4U 1323-619,LMXB-NS
4,4U 1538-522,HMXB-NS


In [10]:
# Add target classification and target type

df2 = df.copy()
nrows = len(df2['SEQID'])
print(nrows)

classification= ['SL' for x in range(nrows) ]
targ_type= ['??' for x in range(nrows) ]


df2.insert(loc=0, column='Classification', value=classification)
df2.insert(loc=2, column='Target Type', value=targ_type)

for target, ttype in zip(obj_type['Confirmed Sources'], obj_type['Type']):
    df2.loc[df2['SL Target'] == target, 'Target Type'] = ttype
    
df2.loc[df2['SL Target'] == '??', 'Classification'] = 'Unk'
df2

713


,Classification,SL Target,Target Type,SEQID,Module,Primary Target,Exposure (s),RA,DEC,Notes,TIME,END_TIME
0,Unk,??,??,10402003002,B,Crab_bkgd_PA330,19232.6,83.1538,21.5296,GR (from Crab?),58156.445245,58156.889690
1,Unk,??,??,10502003002,B,Crab_bkgd_PA330,28264.7,83.1438,21.5151,SL/GR in both,58518.459134,58519.122329
2,Unk,??,??,10601407002,A,N132D,82687.4,81.1972,-69.6945,"Strong on FPMA only, some LMC source",58921.434829,58923.129273
3,Unk,??,??,30001012002,B,NuSTARJ163433m473841,16599.9,248.6712,-47.6363,GR + multiple SL,56374.354944,56374.764666
4,Unk,??,??,30001016002,B,4U1630m47,22071.8,248.5333,-47.3794,"Corner, prob GX 340+0?",56722.955639,56723.486889
...,...,...,...,...,...,...,...,...,...,...,...,...
708,SL,X1908+075,HMXB-NS,90001010002,B,XTE_J1859p083,19838.6,284.7785,8.2742,Faint SL,57111.719528,57112.181333
709,SL,XTE J1550-564?,BHC,30401008002,B,1E_1547d0m5408,90468.5,237.7784,-54.2798,Strong SL in both (Check A),58529.094551,58531.084134
710,SL,XTE J1855-026,HMXB-NS,30360001002,A,Swift_J1843d5m0343,75612.3,280.9001,-3.6847,"Weak SL, second source present",57887.861912,57889.552884
711,SL,XTE J1855-026,HMXB-NS,30360001002,B,Swift_J1843d5m0343,75612.3,280.9001,-3.6847,Weak edge SL in both,57887.861912,57889.552884


In [15]:
tab = Table.from_pandas(df2)

tab.write('test.fits')

In [99]:
print(df2.columns)
arr = df2.to_numpy()

Index(['Classification', 'SL Target', 'Target Type', 'SEQID', 'Module',
       'Primary Target', 'Exposure (s)', 'RA', 'DEC', 'Notes', 'TIME',
       'END_TIME'],
      dtype='object')


In [107]:

# Make the columns the silly way here:

classification = fits.Column(name='CLASSIFICATION', format='10A', array=arr[:, 0])
target = fits.Column(name='SL_SOURCE', format='10A', array=arr[:,1])
ttype = fits.Column(name='TYPE', format='4A', array=arr[:,2])
obsid = fits.Column(name='OBSID', format='12A', array=arr[:,3])
mod = fits.Column(name='MODULE', format='2A', array=arr[:,4])
primary = fits.Column(name='PRIMARY_TARGET', format = '30A', array=arr[:,5])
exposure = fits.Column(name='EXPOSURE', format = 'E', array=[float(e) for e in arr[:,6]])
ra = fits.Column(name='RA', format='E', array =  arr[:, 7])
dec = fits.Column(name='DEC', format='E', array =  arr[:, 8])
tstart = fits.Column(name='TIME', format='E', array = arr[:, 10])
tend = fits.Column(name='TIME_END', format='E', array = arr[:, 11])
notes = fits.Column(name='NOTES', format='30A', array = arr[:, 9])



In [115]:
hdu = fits.BinTableHDU.from_columns([classification, target, obsid, mod, primary, exposure, ra, dec, tstart, tend, notes])

hdu.writeto('fits/straycats.fits', overwrite=True)

In [117]:
straycat = getdata('fits/straycats.fits')
straycat.names

['CLASSIFICATION',
 'SL_SOURCE',
 'OBSID',
 'MODULE',
 'PRIMARY_TARGET',
 'EXPOSURE',
 'RA',
 'DEC',
 'TIME',
 'TIME_END',
 'NOTES']